In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf
from machine_learning_finance import analyze_trades, \
    calc_probabilties_without_lookahead, plot_backtest_analysis, \
    metrics_to_dataframe, create_train_test_windows, make_inverse_env_for
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "XLE"
inverse = "DUG"
file = f"../backtests/backtest_inverse_{symbol}_365_h0.9_l0.1.csv"
period = 365
pd.options.display.max_rows = None


def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()

def analyze_and_graph_range(file, symbol, start, end, df):
    ledger = pd.read_csv(file)
    hist_df, test_df = create_train_test_windows(df, start=start, end=end)
    test_df = calc_probabilties_without_lookahead(test_df, hist_df)
    plot_backtest_analysis(test_df, ledger, inverse=symbol)
    return ledger, test_df
   
def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)
    hist_df = df.head(len(df)-period)
    hist_df = hist_df.tail(period*4)
    df = df.tail(period)
    df = calc_probabilties_without_lookahead(df, hist_df)
    plot_backtest_analysis(df, ledger, inverse=symbol)

    return ledger, df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)
#plot_win_loss(file, metrics)

#print(ledger)

#print(metrics_df)
#df.sort_index(ascending=False)

[*********************100%***********************]  1 of 1 completed


In [3]:
ledger, df2 = analyze_and_graph(file, symbol, 365*4, df)    
#ledger, df2 = analyze_and_graph_range(file, symbol, "03-27-2020", "12-17-2021", df)

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 

merged_df = ledger.merge(df2, left_on='Date', right_index=True, how='left')
merged_df[["Date", "Price", "Product", "Side", "Action","prob_above_trend", "Value"]]

[*********************100%***********************]  1 of 1 completed


,Date,Price,Product,Side,Action,prob_above_trend,Value
0,2021-12-15,55.701502,XLE,long,enter,0.976670,4938.416499
1,2022-05-16,83.739104,XLE,long,exit,0.033719,7464.214641
2,2022-05-16,18.745599,DUG,long,enter,0.033719,7371.870907
3,2022-06-23,23.522901,DUG,long,exit,0.990682,9318.796307
4,2022-06-23,71.366604,XLE,long,enter,0.990682,9203.641311
5,2022-11-01,91.778703,XLE,long,exit,0.025707,11919.244118
6,2022-11-01,12.099800,DUG,long,enter,0.025707,11771.802412
7,2023-05-11,14.544000,DUG,long,exit,0.946117,14253.944964
8,2023-05-11,79.184002,XLE,long,enter,0.946117,14077.974099
9,2023-05-30,78.739599,XLE,long,exit,0.951106,14103.526089


In [ ]:
df2.tail(365)


In [9]:
env = make_inverse_env_for(symbol,
                        inverse,
                        1,
                        period,
                        prob_high=0.9,
                        prob_low=0.1)
env.expert_opinion_df()
print(env.timeseries.loc['2022-08-24'])
print(df2.loc['2022-08-24'])


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Close               0.734599
weighted-volume     0.194735
trend               0.432177
prob_above_trend    0.110829
action              0.000000
Name: 2022-08-24 00:00:00, dtype: float64
Open                8.218000e+01
High                8.330000e+01
Low                 8.181000e+01
Close               8.318000e+01
Adj Close           8.060284e+01
Volume              2.454660e+07
prob_above_trend    1.575455e-02
weighted-volume     2.041786e+09
trend               7.688144e+01
long_entry                   NaN
long_exit                    NaN
short_entry                  NaN
short_exit                   NaN
Name: 2022-08-24 00:00:00, dtype: float64


,Close,weighted-volume,trend,prob_above_trend,action
Date,,,,,
2023-03-20,0.601169,0.193233,0.804384,0.878345,0
2023-03-21,0.666667,0.246643,0.799089,0.771759,0
2023-03-22,0.625030,0.179397,0.793407,0.736225,0
2023-03-23,0.598490,0.177067,0.787647,0.778740,0
2023-03-24,0.605795,0.131951,0.781728,0.847061,0
2023-03-27,0.646214,0.130727,0.836202,0.838577,0
2023-03-28,0.676163,0.089688,0.834711,0.798534,0
2023-03-29,0.703433,0.113733,0.833220,0.716255,0
2023-03-30,0.712199,0.062583,0.831729,0.695412,0
